In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

In [2]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package genesis to
[nltk_data]     /Users/denisdunn/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/denisdunn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/denisdunn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/denisdunn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
df=pd.read_csv('epicurious')

In [4]:
df=df.dropna()

In [5]:
df.head()

,Unnamed: 0,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
1,1,403.0,"['Food Processor', 'Onion', 'Pork', 'Bake', 'B...",2004-08-20 04:00:00.000,This uses the same ingredients found in boudin...,['Combine first 9 ingredients in heavy medium ...,23.0,"['1 1/2 cups whipping cream', '2 medium onions...",18.0,4.375,1439.0,Boudin Blanc Terrine with Red Onion Confit
5,5,948.0,"['Sandwich', 'Food Processor', 'Tomato', 'Kid-...",2004-08-20 04:00:00.000,This recipe can be prepared in 45 minutes or l...,"['Mix basil, mayonnaise and butter in processo...",79.0,['2 1/2 cups (lightly packed) fresh basil leav...,19.0,4.375,1042.0,The Best Blts
8,8,170.0,"['Beef', 'Ginger', 'Sauté', 'Stir-Fry', 'Quick...",2004-08-20 04:00:00.000,Bulgogi,"['Stir together soy sauce, sugar, sesame oil, ...",10.0,"['1/4 cup soy sauce', '1 tablespoon sugar', '2...",7.0,4.375,1272.0,Korean Marinated Beef
9,9,602.0,"['Salad', 'Mustard', 'Potato', 'Picnic', 'Lunc...",2008-10-23 22:24:26.000,Transform your picnic into un pique-nique to r...,['Chop enough parsley leaves to measure 1 tabl...,41.0,"['6 long parsley sprigs, divided', '1 3/4 cups...",23.0,3.750,1696.0,Ham Persillade with Mustard Potato Salad and M...
10,10,256.0,"['Milk/Cream', 'Dairy', 'Side', 'Thanksgiving'...",2004-08-20 04:00:00.000,Simmering the yams fills them with flavor and ...,['Heat oil in heavy large skillet over medium-...,5.0,"['4 teaspoons olive oil', '1/2 cup finely chop...",4.0,3.750,30.0,"Yams Braised with Cream, Rosemary and Nutmeg"


In [6]:
def clean_song(song):
    cleaned_song = []
    for line in song:
        if not '[' in line and  not ']' in line:
            for symbol in ",.?!''\n":
                line = line.replace(symbol, '').lower()
            cleaned_song.append(line)

    return cleaned_song

In [7]:
def tokenize(song):
    joined_song = ' '.join(song)
    tokenized_song = sent_tokenize(joined_song)
    
    return tokenized_song

In [14]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

In [15]:
import string

In [16]:
stopwords_list += list(string.punctuation)

In [17]:
#add words that aren't in the NLTK stopwords list
stopwords_list += ['cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
stopwords_list+=['1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','small','large','medium']

In [18]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [19]:
def process_article(article):
    tokens = word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

In [20]:
def tokenize(song):
    joined_song = ' '.join(song)
    tokenized_song = word_tokenize(joined_song)
    
    return tokenized_song

In [21]:
#df['a'] = df['a'].apply(lambda x: x + 1)
df['Ingredient_join']=df['ingredients'].apply(lambda x:' '.join(x))
 
# df['ingredients'].astype(str).str.cat(df['ingredients'],sep=' ')

In [22]:
#vectorize_list=df['Ingredient_join'].apply(lambda x: process_article(x))

In [23]:
df['processed_words']=df['ingredients'].apply(lambda x: process_article(x))

In [24]:
df['Ingredient_join']=df['processed_words'].apply(lambda x:' '.join(x))

In [25]:
vectorize_list=df['Ingredient_join'].apply(lambda x: process_article(x))

In [26]:
df["Ingredient_join"][1]

"whipping cream onions bay leaves whole cloves garlic clove crushed 1/8 ground nutmeg 'pinch dried thyme crumbled shallots minced butter trimmed boneless center pork loin sinew removed 1-inch chunks well chilled eggs purpose flour tawny port dried currants minced 'lettuce leaves 'cracked peppercorns 'minced parsley 'bay leaves 'french bread baguette slices olive oil red onions halved dried currants red wine vinegar canned chicken broth thyme dried crumbled sugar"

In [27]:
total_vocab = set()
for comment in vectorize_list:
    total_vocab.update(comment)
len(total_vocab)

8042

In [28]:
articles_concat = []
for article in vectorize_list:
    articles_concat += article

In [29]:
#articles_concat

In [30]:
articles_freqdist = FreqDist(articles_concat)
#articles_freqdist.most_common(200)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vectorizer = TfidfVectorizer()

In [33]:
tf_idf_data_train = vectorizer.fit_transform(df['Ingredient_join'])

In [34]:
tf_idf_data_train.shape

(10608, 5431)

In [35]:
non_zero_cols = tf_idf_data_train.nnz / float(tf_idf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tf_idf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 28.41223604826546
Percentage of columns containing 0: 0.9947685074483031


In [42]:
import logging

In [43]:
tf_idf_data_train.shape

(10608, 5431)

In [78]:
texts=df['processed_words']

In [179]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
index_tmpfile = get_tmpfile("index")
#query = [(1, 2), (6, 1), (7, 2)]

#index = Similarity(index_tmpfile, common_corpus, num_features=len(common_dictionary))  # build the index
#similarities = index[query]  # get similarities between the query and all index documents

In [ ]:
>>> from gensim.test.utils import common_corpus, common_dictionary
>>> from gensim.similarities import MatrixSimilarity
>>>
>>> query = [(1, 2), (5, 4)]
>>> index = MatrixSimilarity(common_corpus, num_features=len(common_dictionary))
>>> sims = index[query]

In [79]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
#print(dictionary.token2id)
print(dictionary.values)

<bound method Mapping.values of <gensim.corpora.dictionary.Dictionary object at 0x1a29fded68>>


In [106]:
new_doc= vectorize_list[1]
new_vec = dictionary.doc2bow(new_doc)
print(new_vec[3:6])

[(3, 1), (4, 1), (5, 1)]


In [107]:
trial=new_vec[3:6]

In [81]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [82]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=10608, num_nnz=295740)
295740


In [83]:
from gensim import models
tfidf = models.TfidfModel(corpus)

In [102]:

tfidf

In [85]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [86]:
vec_lsi = lsi[new_vec] 

In [88]:
from gensim import similarities
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

In [89]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [213]:
word2index = {token: token_index for token_index, token in enumerate(w2v.index2word)} 
word2index['hi'] == 30308  # True

NameError: name 'w2v' is not defined

In [210]:
model.wv.vocab.get(query_doc[0]).index

1787

In [215]:
model[1787]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


TypeError: 'int' object is not iterable

In [212]:

corpus[1787].vocab

AttributeError: 'list' object has no attribute 'vocab'

In [90]:
sims = index[vec_lsi]
print(vec_lsi)


[(0, 4.736365050645086), (1, 1.0857745447595581)]


In [92]:
sims = gensim.similarities.Similarity(texts,tf_idf[corpus],
                                      num_features=len(dictionary))
#print(sims)
#print(type(sims))

In [97]:
query_doc = [w.lower() for w in word_tokenize("lasagna")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

['lasagna']
[(3725, 1)]
[(3725, 1.0)]


In [61]:
sims = index[tfidf[vec_lsi]]
print(list(enumerate(sims)))

[(0, 0.9990976), (1, 0.9989896), (2, 0.99075615), (3, 0.9573831), (4, 0.9771822), (5, 0.3724307), (6, 0.87395257), (7, 0.92717206), (8, 0.94329643), (9, 0.57668555), (10, 0.9401987), (11, 0.95449054), (12, 0.8921204), (13, 0.9992929), (14, 0.6469079), (15, 0.6043892), (16, 0.9210047), (17, 0.93413365), (18, 0.909341), (19, 0.7437564), (20, 0.6799887), (21, 0.9409225), (22, 0.95849246), (23, 0.9127256), (24, 0.99979985), (25, 0.6326435), (26, 0.8890683), (27, 0.928672), (28, 0.8064952), (29, 0.6359377), (30, 0.99376047), (31, 0.9283078), (32, 0.58285654), (33, 0.9053169), (34, 0.48425585), (35, 0.5568712), (36, 0.9929355), (37, 0.9389657), (38, 0.95256066), (39, 0.93845457), (40, 0.99897456), (41, 0.99993193), (42, 0.99992883), (43, 0.9384342), (44, 0.96663076), (45, 0.9695411), (46, 0.99387574), (47, 0.9741449), (48, 0.23695804), (49, 0.39125723), (50, 0.9990502), (51, 0.50965106), (52, 0.911705), (53, 0.950181), (54, 0.927956), (55, 0.9955795), (56, 0.6305574), (57, 0.90599936), (58, 

In [115]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

model = Word2Vec(texts, size=20, min_count=1)  # train word-vectors

index = WmdSimilarity(texts, model)
 # Make query.
query = ['broccoli']
sims = index[query]

In [148]:
word_vectors = model.wv.vectors

In [177]:
sims.max()

0.6087932770761569

In [39]:

from sklearn.cluster import KMeans

In [149]:
idx =k_means.fit_predict( word_vectors)

In [169]:
idx.shape

(8084,)

In [152]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
words = list(model.vocab)
for i, word in enumerate(words):  
    print (word + ":" + str(assigned_clusters[i]))

In [157]:
#for cluster in xrange(0,10):

# Print the cluster number

#…………….print "\nCluster %d" % cluster

# Find all of the words for that cluster number, and print them out

words = []

for i in range(0,len(word_centroid_map.values())):

    if( word_centroid_map.values()[i] == cluster ):

        words.append(word_centroid_map.keys()[i])
print(words)

TypeError: 'dict_values' object does not support indexing

In [151]:
plt.scatter(idx[:,0],idx[:1], c=8, s=10)
centers = k_means.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=70)

IndexError: too many indices for array

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [38]:
nb_classifier.fit(tf_idf_data_train, target)
nb_train_preds = nb_classifier.predict(tf_idf_data_train)
nb_test_preds = nb_classifier.predict(tf_idf_data_test)

NameError: name 'target' is not defined

In [37]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier(n_estimators=100)

In [44]:
from sklearn.decomposition.truncated_svd import TruncatedSVD        
pca = TruncatedSVD(n_components=2)                                
X_reduced_train = pca.fit_transform(index) 

NameError: name 'index' is not defined